In [1]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense
from keras.models import Model
from keras.models import model_from_json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [36]:
class Recommender_Model():
	def __init__(self):
		self.dataset = pd.read_csv('../scripts/goodbooks-10k-master/ratings.csv')
		self.book_data = np.array(list(set(self.dataset.book_id)))
		self.books = pd.read_csv('../scripts/goodbooks-10k-master/books.csv')
		
	def create_net(self):

		train, test = train_test_split(self.dataset, test_size=0.2, random_state=42)
		n_users = len(self.dataset.user_id.unique())
		n_books = len(self.dataset.book_id.unique())

		book_input = Input(shape=[1], name="Book-Input")
		book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
		book_vec = Flatten(name="Flatten-Books")(book_embedding)

		user_input = Input(shape=[1], name="User-Input")
		user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
		user_vec = Flatten(name="Flatten-Users")(user_embedding)

		prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
		
		self.model = Model([user_input, book_input], prod)
		self.model.compile('adam', 'mean_squared_error')

		history = self.model.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
	def train(self):
		# serialize model to JSON
		model_json = self.model.to_json()
		with open("model.json", "w") as json_file:
		    json_file.write(model_json)
		# serialize weights to HDF5
		self.model.save_weights("model.h5")
		print("Saved model to disk")

	def predict(self,user_data):

		json_file = open('model.json', 'r')
		loaded_model_json = json_file.read()
		json_file.close()
		loaded_model = model_from_json(loaded_model_json)
		# load weights into new model
		loaded_model.load_weights("model.h5")
		print("Loaded model from disk")

		loaded_model.compile('adam', 'mean_squared_error')

		# Creating dataset for making recommendations for the first user
		if len(user_data) != len(self.book_data):
			return
		user = np.array([1 for i in range(len(self.book_data))])
		#user = np.array(user_data) #[1,1,0,1,0,0,0......0,1,1]
		predictions = loaded_model.predict([user, self.book_data])
		predictions = np.array([a[0] for a in predictions])
		recommended_book_ids = (-predictions).argsort()[:100]

		return self.books[self.books['book_id'].isin(recommended_book_ids)]


In [37]:
mod = Recommender_Model()


In [8]:
mod.create_net()


Epoch 1/5
4781183/4781183 [==============================] - 409s 86us/step - loss: 4.0663
Epoch 2/5
4781183/4781183 [==============================] - 405s 85us/step - loss: 0.7941
Epoch 3/5
4781183/4781183 [==============================] - 416s 87us/step - loss: 0.7716
Epoch 4/5
4781183/4781183 [==============================] - 396s 83us/step - loss: 0.7553
Epoch 5/5
4781183/4781183 [==============================] - 395s 83us/step - loss: 0.7413


In [9]:
#mod.train()

Saved model to disk


In [38]:
output = mod.predict(np.array([1 for i in range(len(mod.book_data))]))

Loaded model from disk


In [39]:
output

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
1097,1098,17407748,17407748,24248331,92,1455520659,9.781456e+12,Nicholas Sparks,2012.0,The Longest Ride,...,71276,88683,8692,1011,3435,15361,31569,37307,https://images.gr-assets.com/books/1367536453m...,https://images.gr-assets.com/books/1367536453s...
1378,1379,13136,13136,1835674,103,446696633,9.780447e+12,"James Patterson, Andrew Gross",2002.0,2nd Chance,...,68834,75300,2128,391,2580,17641,29784,24904,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
2679,2680,52309,52309,598905,834,743255062,9.780743e+12,"Benjamin Franklin, Lewis Leary",1791.0,NaN,...,33061,43113,2159,1482,2925,10378,15150,13178,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
3080,3081,12647642,12647642,17760917,13,NaN,NaN,Tina Reber,2012.0,Love Unrehearsed,...,42280,42818,1828,572,1336,5836,13736,21338,https://images.gr-assets.com/books/1347242126m...,https://images.gr-assets.com/books/1347242126s...
3122,3123,2159,2159,1734580,71,452287081,9.780452e+12,John Perkins,2004.0,Confessions of an Economic Hit Man,...,19726,26245,3036,663,1851,6174,10056,7501,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
3246,3247,3977,3977,1882574,58,553572946,9.780554e+12,Dan Simmons,1996.0,Endymion,...,30664,33572,969,273,1201,5556,12997,13545,https://images.gr-assets.com/books/1329611385m...,https://images.gr-assets.com/books/1329611385s...
3383,3384,12351649,12351649,17912072,15,NaN,9.780983e+12,R.L. Mathewson,2011.0,Perfection,...,50146,54721,2483,612,1893,8852,19987,23377,https://images.gr-assets.com/books/1406578931m...,https://images.gr-assets.com/books/1406578931s...
3393,3394,2177236,2177236,14592071,96,525950608,9.780526e+12,Harlan Coben,2008.0,Hold Tight,...,30180,33433,2241,256,1173,8187,14689,9128,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
3489,3490,20767918,20767918,40105061,7,1455521191,9.781456e+12,David Baldacci,2014.0,The Escape,...,18073,29683,2317,265,771,4660,12864,11123,https://images.gr-assets.com/books/1404163742m...,https://images.gr-assets.com/books/1404163742s...
3518,3519,31196,31196,2095259,152,1400034205,9.781400e+12,W. Somerset Maugham,1944.0,The Razor's Edge,...,25267,28640,2172,222,910,4563,10773,12172,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [19]:
mod.books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [28]:
len(mod.book_data)

10000